# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.10249


extracting tarball to tmp_2301.10249...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.10251


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.10249/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Retrieving document from  https://arxiv.org/e-print/2301.10717


extracting tarball to tmp_2301.10717...

 done.


Retrieving document from  https://arxiv.org/e-print/2301.10728


/tmp/ipykernel_2055/4030337529.py:34: LatexWarning: 2301.10728 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10496-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10496) | **Understanding the TeV $γ$-ray emission surrounding the young  massive star cluster Westerlund 1**  |
|| Lucia K. Härer, et al. -- incl., <mark>Lars Mohrmann</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *Accepted for publication in Astronomy & Astrophyics*|
|**Abstract**| Context: Young massive star clusters (YMCs) have come increasingly into the focus of discussions on the origin of galactic cosmic rays (CRs). The proposition of CR acceleration inside superbubbles (SBs) blown by the strong winds of these clusters avoids issues faced by the standard paradigm of acceleration at supernova remnant shocks. Aims: We provide an interpretation of the latest TeV $\gamma$-ray observations of the region around the YMC Westerlund 1 taken with the High Energy Stereoscopic System (H.E.S.S.) in terms of diffusive shock acceleration at the cluster wind termination shock, taking into account the spectrum and morphology of the emission. As Westerlund 1 is a prototypical example of a YMC, such a study is relevant to the general question about the role of YMCs for the Galactic CR population. Methods: We generate model $\gamma$-ray spectra, characterise particle propagation inside the SB based on the advection, diffusion, and cooling timescales, and constrain key parameters of the system. We consider hadronic emission from proton-proton interaction and subsequent pion decay and leptonic emission from inverse Compton scattering on all relevant photon fields, including the CMB, diffuse and dust-scattered starlight, and the photon field of Westerlund 1 itself. The effect of the magnetic field on cooling and propagation is discussed. Klein-Nishina effects are found to be important in determining the spectral evolution of the electron population. Results: A leptonic origin of the bulk of the observed $\gamma$-rays is preferable. The model is energetically plausible, consistent with the presence of a strong shock, and allows for the observed energy-independent morphology. The hadronic model faces two main issues: confinement of particles to the emission region and an unrealistic energy requirement. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10666-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10666) | **The interplay of internal and external processes in the buildup of disk  galaxies: thick-disk star formation histories in AURIGA simulations**  |
|| <mark>Francesca Pinna</mark>, Daniel Walo-Martín, Robert J. J. Grand |
|*Appeared on*| *2023-01-26*|
|*Comments*| *3 pages, one figure, Proceedings IAU Symposium S373*|
|**Abstract**| Recent integral-field spectroscopy observations have revealed that thick- and thin-disk star-formation histories are regulated by the interplay of internal and external processes. We analyze stellar-population properties of 24 spiral galaxies from the AURIGA zoom-in cosmological simulations, to offer a more in-depth interpretation of observable properties. We present edge-on maps of stellar age, metallicity and [Mg/Fe] abundance, and we extract the star-formation and chemical-evolution histories of thin and thick disks. Both show signs of the interplay between internal chemical enrichment and gas and star accretion. Thick disks show particularly complex stellar populations, including an in-situ component, formed from both slowly enriched and accreted more pristine gas, and an additional significant fraction of ex-situ stars. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10717-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10717) | **Strong (Hb + [OIII]) and Ha emitters at redshift z ~ 7-8 unveiled with  JWST/NIRCam and MIRI imaging in the Hubble eXtreme Deep Field (XDF)**  |
|| P. Rinaldi, et al. -- incl., <mark>A. Alonso-Herrero</mark>, <mark>F. Walter</mark>, <mark>S. Bosman</mark>, <mark>D. Langeroodi</mark>, <mark>R. Meyer</mark>, <mark>M. Guedel</mark>, <mark>Th. Henning</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *15 pages, 9 figures. Submitted to ApJ*|
|**Abstract**| The JWST is revolutionizing the study of high-redshift galaxies by providing for the first time a high-sensitivity view of the early Universe at infrared wavelengths, both with its Near Infrared Camera (NIRCam) and Mid Infrared Instrument (MIRI). In this paper, we make use of medium and broad-band NIRCam imaging, as well as ultra-deep MIRI 5.6 microns imaging, in the Hubble eXtreme Deep Field (XDF) to identify prominent line emitters at z ~ 7-8. Out of a total of 58 galaxies at z ~ 7-8, we find 18 robust candidates (~31%) for prominent (Hb + [OIII]) emitters, based on their enhanced fluxes in the F430M and F444W filters, with rest-frame EW(Hb + [OIII]) ~ 87 - 2100 A. Among these emitters, 16 lie on the MIRI coverage area and 12 show a clear flux excess at 5.6 microns, indicating the simultaneous presence of a prominent Ha emission line with rest-frame EW(Ha) ~ 200 - 3000 A. This is the first time that Ha emission can be detected in individual galaxies at z>7. The Ha line, when present, allows us to separate the contributions of the Hb and [OIII] emission lines to the (Hb + [OIII]) complex and derive Ha-based star formation rates (SFRs). We find that in some cases [OIII]/Hb > 1, suggesting low metallicities, but a few have [OIII]/Hb < 1, so the NIRCam flux excess is mainly driven by Hb. The vast majority of prominent line emitters are very young starbursts or galaxies on their way to/from the starburst cloud. They make for a cosmic SFR density log10(SFRD_Ha / Msun yr^-1 Mpc^-3) ~ 2.35, which is about a third of the total value at z ~ 7-8. Therefore, the strong Ha emitters likely had an important role in reionization. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10533-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10533) | **Capturing the lowest luminosity state of the Supergiant Fast X-ray  Transient XTE J1739-302**  |
|| <mark>Lara Sidoli</mark>, Gabriele Ponti, Vito Sguera, Paolo Esposito |
|*Appeared on*| *2023-01-26*|
|*Comments*| *10 pages, 9 Figures, 2 Tables; accepted for publication in Astronomy & Astrophysics on 24 January 2023*|
|**Abstract**| We report here on the results of the analysis of Chandra, XMM-Newton and NuSTAR recent observations of the Supergiant Fast X-ray Transient XTEJ1739-302. The source was caught in a low X-ray luminosity state, from a few $10^{31}$ to $10^{34}$ erg/s (0.5-10 keV). In particular, a very low X-ray luminosity was captured during an XMM-Newton observation performed in October 2022, at a few $10^{31}$ erg/s (0.5-10 keV), never observed before in XTEJ1739-302. The XMM-Newton spectrum could be well fitted either by an absorbed, steep power law model (photon index of 3.5) or by a collisionally-ionized diffuse gas with a temperature of 0.7 keV, very likely produced by shocks in the supergiant donor wind. These observations covered different orbital phases, but all appear compatible with the low luminosity level expected from the orbital INTEGRAL light curve. The absorbing column density is variable in the range $10^{22}-10^{23}$ cm$^{-2}$. A broad-band X-ray spectrum could be investigated at $10^{34}$ erg/s (0.5-30 keV) for the first time in XTEJ1739-302 with not simultaneous (but at similar orbital phases) Chandra and NuSTAR data, showing a power law spectral shape with a photon index of about 2.2 and an absorbing column density of $\sim$$10^{23}$ cm$^{-2}$. Remarkably, owing to the XMM-Newton observation, the amplitude of the X-ray variability has increased to five orders of magnitude, making XTEJ1739-302 one of the most extreme SFXTs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10613) | **Hubble Space Telescope imaging of the compact elliptical galaxy M32  reveals a dearth of carbon stars**  |
|| O. C. Jones, et al. -- incl., <mark>M. Meixner</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *10 pages, 7 figures, submitted MNRAS 7/12/22*|
|**Abstract**| We present new {\em Hubble Space Telescope} WFC3/IR medium-band photometry of the compact elliptical galaxy M32, chemically resolving its thermally pulsating asymptotic giant branch stars. We find 2829 M-type stars and 57 C stars. The carbon stars are likely contaminants from M31. If carbon stars are present in M32 they are so in very low numbers. The uncorrected C/M ratio is 0.020 $\pm$ 0.003; this drops to less than 0.007 after taking into account contamination from M31. As the mean metallicity of M32 is just below solar, this low ratio of C to M stars is unlikely due to a metallicity ceiling for the formation of carbon stars. Instead, the age of the AGB population is likely to be the primary factor. The ratio of AGB to RGB stars in M32 is similar to that of the inner disc of M31 which contain stars that formed 1.5--4 Gyr ago. If the M32 population is at the older end of this age then its lack of C-stars may be consistent with a narrow mass range for carbon star formation predicted by some stellar evolution models. Applying our chemical classifications to the dusty variable stars identified with {\em Spitzer}, we find that the x-AGB candidates identified with {\em Spitzer} are predominately M-type stars. This substantially increases the lower limit to the cumulative dust-production rate in M32 to $>$ 1.97 $\times 10^{-5}$ ${\rm M}_{\odot} \, {\rm yr}^{-1}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10614-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10614) | **Optical and near-infrared stellar activity characterization of the early  M dwarf Gl~205 with SOPHIE and SPIRou**  |
|| P. Cortes-Zuleta, et al. -- incl., <mark>B. Klein</mark>, <mark>N. Heidari</mark>, <mark>G. Hebrard</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *42 pages, 24 figures. Accepted for publication in A&A*|
|**Abstract**| The stellar activity of M dwarfs is the main limitation for discovering and characterizing exoplanets orbiting them since it induces quasi-periodic RV variations. We aim to characterize the magnetic field and stellar activity of the early, moderately active, M dwarf Gl205 in the optical and nIR domains. We obtained high-precision quasi-simultaneous spectra in the optical and nIR with the SOPHIE spectrograph and SPIRou spectropolarimeter between 2019 and 2022. We computed the RVs from both instruments and the SPIRou Stokes V profiles. We used ZDI to map the large-scale magnetic field over the time span of the observations. We studied the temporal behavior of optical and nIR RVs and activity indicators with the Lomb-Scargle periodogram and a quasi-periodic GP regression. In the nIR, we studied the equivalent width of Al I, Ti I, K I, Fe I, and He I. We modeled the activity-induced RV jitter using a multi-dimensional GP regression with activity indicators as ancillary time series. The optical and nIR RVs have similar scatter but nIR shows a more complex temporal evolution. We observe an evolution of the magnetic field topology from a poloidal dipolar field in 2019 to a dominantly toroidal field in 2022. We measured a stellar rotation period of Prot=34.4$\pm$0.5 d in the longitudinal magnetic field. Using ZDI we measure the amount of latitudinal differential rotation (DR) shearing the stellar surface yielding rotation periods of Peq=32.0$\pm$1.8 d at the stellar equator and Ppol=45.5$\pm$0.3 d at the poles. We observed inconsistencies in the activity indicators' periodicities that could be explained by these DR values. The multi-dimensional GP modeling yields an RMS of the RV residuals down to the noise level of 3 m/s for both instruments, using as ancillary time series H$\alpha$ and the BIS in the optical, and the FWHM in the nIR. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10615-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10615) | **Quakes of Compact Stars**  |
|| Ruipeng Lu, et al. -- incl., <mark>Shenjian Zhang</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *13 pages, 7 figures; MNRAS paper accepted*|
|**Abstract**| Glitches are commonly observed for pulsars, which are explained by various mechanisms. One hypothesis attributes the glitch effect to the instantaneous moment of inertia change of the whole star caused by a starquake, which is similar to earthquakes caused by fast dislocation occurring on planar faults for the static stress, though the quake-induced dynamics responsible for glitch (superfluid vortex vs. pure starquake) remains still unknown. However, a theoretical model to quantitatively explain the stress loading, types of starquakes, and co-seismic change of moment of inertia is rarely discussed. In this study, we incorporate elastic deformation theories of earthquakes into the starquake problems. We compute the field of stress loading associated with rotation deceleration and determine the optimal type of starquakes at various locations. Two types of pulsar structure models, i.e. neutron and strangeon star models, are included in the computation and their differences are notable. Our calculation shows that the observed glitch amplitude can be explained by the starquakes in the strangeon star model, though the required scaled starquake magnitude is much larger than that occurred on the Earth. We further discuss the possibility to compute the energy budget and other glitch phenomena using the starquake model in the elastic medium framework. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.10728-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.10728) | **CdTe Spectroscopic-Imager Measurements with Bent Crystals for Broad Band  Laue Lenses**  |
|| N. Auricchio, et al. -- incl., <mark>O. Limousin</mark> |
|*Appeared on*| *2023-01-26*|
|*Comments*| *7 pages, 22 figures, 2022 IEEE Nuclear Science Symposium, Medical Imaging Conference and Room Temperature Semiconductor Detector Conference (NSS/MIC/RTSD)*|
|**Abstract**| In astrophysics, several key questions in the hard X soft Gamma-ray range (above 100 keV) require sensitivity and angular resolution that are hardly achievable with current technologies. Therefore, a new kind of instrument able to focus hard X and gamma-rays is essential. Broad band Laue lenses seem to be the only solution to fulfil these requirements, significantly improving the sensitivity and angular resolution of the X and gamma-ray telescopes. This type of high-energy optics will require highly performing focal plane detectors in terms of detection efficiency, spatial resolution, and spectroscopy. This paper presents the results obtained in the project 'Technological Readiness Increase for Laue Lenses (TRILL)' framework using a Caliste-HD detector module. This detector is a pixel spectrometer developed at CEA (Commissariat a Energie Atomique, Saclay, France). It is used to acquire spectroscopic images of the focal spot produced by Laue Lens bent crystals under a hard X-ray beam at the LARIX facility (University of Ferrara, Italy). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.10496.md
    + _build/html/tmp_2301.10496/./fig/HESSband_compare_paper.png
    + _build/html/tmp_2301.10496/./fig/tcool_paper.png
    + _build/html/tmp_2301.10496/./fig/HESS_W1.png
exported in  _build/html/2301.10666.md
    + _build/html/tmp_2301.10666/./AU_5_sfh127_thetaY0_SN30_minNstars2_run0_thickexsitu_normgal_talk.png
exported in  _build/html/2301.10717.md
    + _build/html/tmp_2301.10717/./Age_VS_EW_OIII.png
    + _build/html/tmp_2301.10717/./MASS_VS_EW_OIII.png
    + _build/html/tmp_2301.10717/./EW_OIII_vs_EW_Halpha.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\hess}{H.E.S.S.}$
$\newcommand{\src}{Westerlund~1}$
$\newcommand{\kms}{ \mathrm{km} \mathrm{s}^{-1}}$
$\newcommand{\Msyr}{ \mathrm{M}_\odot \mathrm{yr}^{-1}}$
$\newcommand{\ergs}{ \mathrm{erg} \mathrm{s}^{-1}}$
$\newcommand{\ergqcm}{ \mathrm{erg} \mathrm{cm}^{-3}}$
$\newcommand{\eVqcm}{ \mathrm{eV} \mathrm{cm}^{-3}}$
$\newcommand{\pqcm}{ \mathrm{cm}^{-3}}$
$\newcommand{\Msun}{ \mathrm{M}_\odot}$
$\newcommand{\Lw}{L_\mathrm{w}}$
$\newcommand{\arraystretch}{1.2}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\blue$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\hess$}{H.E.S.S.}$
$\newcommand{$\src$}{Westerlund~1}$
$\newcommand{$\kms$}{ \mathrm{km} \mathrm{s}^{-1}}$
$\newcommand{$\Msyr$}{ \mathrm{M}_\odot \mathrm{yr}^{-1}}$
$\newcommand{$\ergs$}{ \mathrm{erg} \mathrm{s}^{-1}}$
$\newcommand{$\ergqcm$}{ \mathrm{erg} \mathrm{cm}^{-3}}$
$\newcommand{$\eVqcm$}{ \mathrm{eV} \mathrm{cm}^{-3}}$
$\newcommand{$\pqcm$}{ \mathrm{cm}^{-3}}$
$\newcommand{$\Msun$}{ \mathrm{M}_\odot}$
$\newcommand{$\Lw$}{L_\mathrm{w}}$
$\newcommand{$\arraystretch$}{1.2}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.10496-b31b1b.svg)](https://arxiv.org/abs/2301.10496)<mark>Appeared on: 2023-01-26</mark> - _Accepted for publication in Astronomy & Astrophyics_

</div>
<div id="authors">

Lucia K. Härer, et al. -- incl., <mark><mark>Lars Mohrmann</mark></mark>

</div>
<div id="abstract">

**Abstract:** Young massive star clusters (YMCs) have come increasingly into the focus of discussions on the origin of galactic cosmic rays (CRs). The proposition of CR acceleration inside superbubbles (SBs) blown by the strong winds of these clusters avoids issues faced by the standard paradigm of acceleration at supernova remnant shocks.We provide an interpretation of the latest TeV$\gamma$-ray observations   of the region around the YMC$\src$taken with the High Energy Stereoscopic System ($\hess$)   in terms of diffusive shock acceleration at the cluster wind termination shock, taking into account the spectrum and morphology of the emission. As$\src$is a prototypical example of a YMC, such a study is relevant to the general question about the role of YMCs for the Galactic CR population.We generate model$\gamma$-ray spectra, characterise particle propagation inside the SB based on the advection, diffusion, and cooling timescales, and constrain key parameters of the system. We consider hadronic emission from proton-proton interaction and subsequent pion decay and leptonic emission from inverse Compton scattering on all relevant photon fields, including the CMB, diffuse and dust-scattered starlight, and the photon field of$\src$itself. The effect of the magnetic field on cooling and propagation is discussed. Klein-Nishina effects are found to be important in determining the spectral evolution of the electron population.A leptonic origin of the bulk of the observed$\gamma$-rays is preferable. The model is energetically plausible, consistent with the presence of a strong shock, and allows for the observed energy-independent morphology. The hadronic model faces two main issues: confinement of particles to the emission region and an unrealistic energy requirement.

</div>

<div id="div_fig1">

<img src="tmp_2301.10496/./fig/HESSband_compare_paper.png" alt="Fig3" width="100%"/>

**Figure 3. -** The best models for the spectrum of HESS J1646$-$458 (data taken from \citetalias{Aharonian22}). Standard parameters are assumed (see Table \ref{tab:pars}). The injected particle spectrum is a power law with an exponential cut-off, where $\alpha_\mathrm{inj}$ is the index and $E_\mathrm{cutoff}$ the cut-off energy. The \texttt{GAMERA} library is used to calculate the cooled particle spectrum and $\gamma$-ray production. (*fig:hessband*)

</div>
<div id="div_fig2">

<img src="tmp_2301.10496/./fig/tcool_paper.png" alt="Fig7" width="100%"/>

**Figure 7. -** \textit{Left:} electron cooling times for the default case (see Table \ref{tab:pars}). The broad grey line is the sum of all the components shown in colour. For a description of the photon fields for IC scattering see the text. \textit{Right:} range of plausible cooling times for synchrotron radiation (orange), bremsstrahlung (green), and the sum of all IC components (blue), resulting from a cluster photon field energy density of $U_\mathrm{cl} = 3\mbox{--}180$\eVqcm$$, a density inside the superbubble of $n_\mathrm{int} = 0.02\mbox{--}0.1$\pqcm$$, an effective cluster temperature of $T_\mathrm{eff} = 10{,}000\mbox{--}50{,}000 $K, a magnetic field of $B = 1\mbox{--}10\mu$G, and an enhancement of the diffuse stellar and dust emission by a factor 1--3. The enhancement of the diffuse component is motivated by the proximity to the cluster and increased dust density in the region, compared to the standard ISM. The inset shows the behaviour in the TeV-band in more detail highlighting the loss-time scaling with energy. The default case (broad grey line) shows a behaviour that would require diffusion close to the Kraichnan regime ($D\sim E^{1/2}$) to reproduce energy-independent morphology. (*fig:tcool*)

</div>
<div id="div_fig3">

<img src="tmp_2301.10496/./fig/HESS_W1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Map of HESS J1646$-$458 taken from \citetalias{Aharonian22}, overlayed with the termination shock position predicted for the parameters indicated in Eq. \ref{eq:rts}. The star marks the position of $\src$. The dashed grey line traces the galactic plane. (*fig:hess_w1*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# The interplay ofinternal and external processesin the buildup of disk galaxies: thick-disk star formation historiesin AURIGA simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.10666-b31b1b.svg)](https://arxiv.org/abs/2301.10666)<mark>Appeared on: 2023-01-26</mark> - _3 pages, one figure, Proceedings IAU Symposium S373_

</div>
<div id="authors">

<mark><mark>Francesca Pinna</mark></mark>, Daniel Walo-Martín, Robert J. J. Grand

</div>
<div id="abstract">

**Abstract:** Recent integral-field spectroscopy observations have revealed that thick- and thin-disk star-formation histories are regulated by the interplay of internal and external processes.We analyze stellar-population properties of 24 spiral galaxies from the AURIGA zoom-in cosmological simulations, to offer a more in-depth interpretation of observable properties.We present edge-on maps of stellar age, metallicity and[Mg/Fe]abundance,and we extract the star-formation and chemical-evolution histories of thin and thick disks. Both show signs of the interplay between internal chemical enrichment and gas and star accretion. Thick disks show particularly complex stellar populations, including an in-situ component, formed from both slowly enriched and accreted more pristine gas, and an additional significant fraction of ex-situ stars.\keywords{techniques: spectroscopic, galaxies: evolution, galaxies: structure}

</div>

<div id="div_fig1">

<img src="tmp_2301.10666/./AU_5_sfh127_thetaY0_SN30_minNstars2_run0_thickexsitu_normgal_talk.png" alt="Fig1" width="100%"/>

**Figure 1. -** Star-formation history of the thick disk in AU5, color-coded by total metallicity (top panels) and [Mg/Fe] abundance (bottom panels). Left panels include all stars, while middle and right panels include respectively only stars formed in situ or ex situ, indicating their corresponding thick-disk mass fraction on top of each column.  (*fig3*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\gsim}{{\;\raise0.3ex\hbox{>\kern-0.75em\raise-1.1ex\hbox{\sim}}\;}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\gsim$}{{\;\raise0.3ex\hbox{>\kern-0.75em\raise-1.1ex\hbox{\sim}}\;}}$</div>



<div id="title">

#  unveiled with JWST/NIRCam and MIRI imaging in the Hubble eXtreme Deep Field (XDF)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.10717-b31b1b.svg)](https://arxiv.org/abs/2301.10717)<mark>Appeared on: 2023-01-26</mark> - _15 pages, 9 figures. Submitted to ApJ_

</div>
<div id="authors">

P. Rinaldi, et al. -- incl., <mark><mark>A. Alonso-Herrero</mark></mark>, <mark><mark>F. Walter</mark></mark>, <mark><mark>S. Bosman</mark></mark>, <mark><mark>D. Langeroodi</mark></mark>, <mark><mark>R. Meyer</mark></mark>, <mark><mark>M. Guedel</mark></mark>, <mark><mark>Th. Henning</mark></mark>

</div>
<div id="abstract">

**Abstract:** The\textit{JWST}is revolutionizing the study of high-redshift galaxies by providing for the first time a high-sensitivity view of the early Universe at infrared wavelengths, both with its Near Infrared Camera (NIRCam) and Mid Infrared Instrument (MIRI). In this paper we make use of medium and broad-band NIRCam imaging, as well as ultra-deep MIRI$5.6 \rm \mu m$imaging, in the Hubble eXtreme Deep Field (XDF) to identify prominent line emitters at$z\simeq 7-8$. Out of a total of 58 galaxies at$z\simeq 7-8$, we find 18 robust candidates ($\simeq$31\%) for prominent (H$\beta$+[OIII]) emitters, based on their enhanced fluxes in the F430M and F444W filters, with rest-frame EW(H$\beta$+[OIII])$\simeq 87 - 2100   \rm \AA$.  Among these emitters, 16 lie on the MIRI coverage area and 12 show a clear flux excess at$5.6   \rm \mu m$, indicating the simultaneous presence of a prominent H$\alpha$emission line with rest-frame EW(H$\alpha$)$\simeq 200-3000   \rm \AA$. This is the first time that  H$\alpha$emission can be detected in individual galaxies at$z>7$. The H$\alpha$line, when present,  allows us to separate the  contributions of the H$\beta$and[OIII]emission lines to the (H$\beta$+[OIII]) complex and derive H$\alpha$-based star formation rates (SFRs). We find that in some cases[OIII]/H$\beta > 1$, suggesting low metallicities, but a few have[OIII]/H$\beta < 1$, so the NIRCam flux excess is mainly driven by H$\beta$. The vast majority of prominent line emitters are very young starbursts or galaxies on their way to/from the starburst cloud. They make for a cosmic SFR density$\rm log_{10}(\rho_{SFR_{H\alpha}}/ \rm M_\odot   yr^{-1}   Mpc^{-3}) \simeq -2.35$, which is about a third of the total value at$z\simeq 7-8$. Therefore, the strong H$\alpha$emitters likely had an important role in reionization.

</div>

<div id="div_fig1">

<img src="tmp_2301.10717/./Age_VS_EW_OIII.png" alt="Fig1" width="100%"/>

**Figure 1. -** Age versus (H$\beta$ + [OIII]) rest-frame EW. The gray triangles refer to the upper limits we estimated for all those galaxies we classified as “non-emitters” during our selection. We also report data points from \citet{Endsley_2021} to make a comparison with the recent literature at high redshift. The curves refer to the evolution of the H$\beta$ rest-frame EW as a function of the age. We plot these tracks for the two metallicty (solar and sub-solar) we have considered in our work and for two different SFHs. A clear anti-correlation between rest-frame EW and age is evident in this plot, as found at lower redshifts from \citet{Reddy_2018}. The same has been found in \citet{Endsley_2021} as well. (*Fig_EWage*)

</div>
<div id="div_fig2">

<img src="tmp_2301.10717/./MASS_VS_EW_OIII.png" alt="Fig2" width="100%"/>

**Figure 2. -** Stellar mass versus (H$\beta$ + [OIII]) rest-frame EW. The gray triangles refer to the upper limits we estimated for all those galaxies we classified as “non-emitters” during our selection. Also in this case, we report data points from \citet{Endsley_2021} to make a comparison with the recent literature at high redshift. The rest-frame EW broadly anti-correlates with stellar mass, similarly to what has been reported by \citet{Reddy_2018} and \citet{Endsley_2021} at lower redshifts. (*Fig_EWstm*)

</div>
<div id="div_fig3">

<img src="tmp_2301.10717/./EW_OIII_vs_EW_Halpha.png" alt="Fig3" width="100%"/>

**Figure 3. -** (H$\beta$ + [OIII]) rest-frame EW versus (H$\alpha$ + [NII] + [SII]) rest-frame EW. Here we colour-coded our data points for the [OIII]$\lambda$5007/H$\beta$ flux ratio. There are some data points there are well above the identity line and show a [OIII]$\lambda$5007/H$\beta$ which is dominated by H$\beta$. However, there are a few galaxies that lie above the identity line that show [OIII]$\lambda$5007/H$\beta$$>$ 1 which can be explained by a non-negligible dust extinction.  (*Fig_6*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

183  publications in the last 7 days.
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers